# 🎵 Spotify Analytics Pipeline - Démonstration Complète

Ce notebook documente et démontre les deux pipelines ETL du projet d'analyse de données Spotify.

## 📋 Table des Matières

1. [Introduction et Configuration](#introduction)
2. [Architecture de l'application](#architecture)
3. [Modélisation conceptuelle des données](#modelisation)
4. [Pipeline 1 : CSV → Oracle → XML → HTML](#pipeline1)
5. [Pipeline 2 : XML → XSD → JSON → MongoDB](#pipeline2)
6. [Conclusion](#conclusion)

---

## Systèmes de Gestion de Bases de Données Utilisés

Ce projet exploite deux types de SGBD complémentaires, illustrant les approches relationnelle et NoSQL de la gestion de données.

### Oracle Database

<div align="center">
  <img src="./images/oracle_logo.png" alt="Oracle Database" width="200">
</div>

**Type** : Base de données relationnelle (SQL)  
**Usage dans le projet** : Stockage structuré et normalisé des données Spotify (8 tables interconnectées)  
**Avantages** : Intégrité référentielle, transactions ACID, requêtes SQL complexes

---

### MongoDB

<div align="center">
  <img src="./images/mongodb_logo.png" alt="MongoDB" width="200">
</div>

**Type** : Base de données NoSQL orientée documents  
**Usage dans le projet** : Stockage flexible des données JSON converties depuis XML  
**Avantages** : Schéma flexible, scalabilité horizontale, performances élevées pour données semi-structurées

---

<div align="center">
  <table>
    <tr>
      <td align="center" width="50%">
        <img src="./images/oracle_logo.png" width="150"><br>
        <strong>Oracle Database</strong><br>
        <em>Approche relationnelle</em>
      </td>
      <td align="center" width="50%">
        <img src="./images/mongodb_logo.png" width="150"><br>
        <strong>MongoDB</strong><br>
        <em>Approche NoSQL</em>
      </td>
    </tr>
  </table>
</div>

## 1. Introduction et Configuration <a id="introduction"></a>

L'objectif de ce projet est de concevoir et mettre en œuvre un pipeline complet de traitement de données, depuis leur acquisition jusqu'à leur exploitation finale. Ce travail vise à démontrer concrètement les différentes étapes et processus nécessaires pour transformer des données brutes en informations exploitables et structurées.

Dans le cadre de ce projet, nous avons développé deux pipelines distincts et complémentaires permettant d'illustrer l'ensemble du cycle de vie des données.


**Pipeline 1 : Relationnel (Oracle)**
```
CSV → Normalisation → Oracle → XML → Validation DTD → XSLT → HTML Dashboard
```

**Pipeline 2 : NoSQL (MongoDB)**
```
XML → Génération XSD → Validation XSD → XSLT → JSON → MongoDB
```

## 2. Architecture de l'application <a id="architecture"></a>

Le projet est organisé selon une architecture modulaire et structurée, facilitant la maintenance et l'évolutivité du code.

### Structure des répertoires
```
DBA_Spotify/
├── main.py                      # Point d'entrée principal (2 pipelines)
├── spotify_pipeline_demo.ipynb  # Jupyter Notebook de démonstration (NOUVEAU)
├── requirements.txt             # Dépendances Python
├── configs/
│   └── config.py               # Configuration (Oracle, MongoDB, chemins)
├── DB/
│   ├── db_manager.py           # Gestionnaire Oracle
│   ├── mongodb_manager.py      # Gestionnaire MongoDB (NOUVEAU)
│   └── db_schema.py            # Schéma des tables Oracle
├── services/
│   ├── data_processor.py       # Traitement et normalisation CSV
│   ├── xml_exporter.py         # Export vers XML
│   ├── dtd_creator.py          # Génération de DTD
│   ├── dtd_validator.py        # Validation DTD
│   ├── xsd_creator.py          # Génération de XSD (NOUVEAU)
│   ├── xsd_validator.py        # Validation XSD (NOUVEAU)
│   ├── json_converter.py       # Conversion XML → JSON (NOUVEAU)
│   └── xslt_transformer.py     # Transformation XSLT → HTML
└── data/
    ├── input/
    │   ├── high_popularity_spotify_data.csv    # Données source
    │   ├── spotify_transform.xslt              # Template XSLT → HTML
    │   └── spotify_to_json.xslt                # Template XSLT → JSON (NOUVEAU)
    └── output/
        ├── spotify_data_export.xml             # XML généré
        ├── spotify_data.dtd                    # DTD généré
        ├── spotify_data.xsd                    # XSD généré (NOUVEAU)
        ├── spotify_data.json                   # JSON généré (NOUVEAU)
        └── spotify_data.html                   # Dashboard HTML
```

### Modules principaux

- `data_processor.py` : Extraction, nettoyage et normalisation des données CSV
- `db_schema.py` : Définition du schéma relationnel (8 tables)
- `db_manager.py` : Gestion des connexions et insertions dans Oracle
- `xml_exporter.py` : Export des données Oracle en format XML structuré
- `dtd_creator.py` / `dtd_validator.py` : Création et validation de la DTD
- `xslt_transformer.py` : Transformation XML → HTML via XSLT
- `configs.py` : Paramètres de connexion à la base de données et chemins des fichiers
- `main.py` : Orchestration complète des deux pipelines

Cette architecture modulaire permet une séparation claire des responsabilités et facilite les tests unitaires et la maintenance du code.



## 3. Modélisation conceptuelle des données <a id="modelisation"></a>

Sur la base de l'analyse des données CSV, nous avons élaboré le Modèle Conceptuel de Données suivant, mettant en évidence 8 entités principales et leurs cardinalités :

![MCD Spotify](./images/Spotify_modelisation.png)

*Figure 1 : Modèle Conceptuel de Données représentant les entités et associations du système Spotify*

Ce MCD met en évidence 8 entités principales interconnectées par des associations de cardinalités variées (1:N, N:M). Il nous servira de base pour la conception du schéma relationnel normalisé.

### Imports et Configuration

Le code Python suivant importe les modules nécessaires à l'exécution des deux pipelines du projet.

In [ ]:
# Imports standards
import sys
import os
import json
from pathlib import Path
import subprocess
import pandas as pd
from datetime import datetime

# Ajouter le répertoire du projet au PYTHONPATH
project_root = Path.cwd()
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

print(f"📁 Répertoire de travail : {project_root}")
print(f"📅 Date : {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

In [ ]:
# Imports des modules du projet
try:
    from DB.db_manager import DatabaseManager
    from DB.mongodb_manager import MongoDBManager
    from services.data_processor import preprocess_csv
    from services.xml_exporter import export_to_xml
    from services.dtd_creator import create_spotify_dtd
    from services.dtd_validator import validate_xml_with_dtd
    from services.xsd_creator import create_spotify_xsd
    from services.xsd_validator import validate_xml_with_xsd
    from services.json_converter import convert_xml_to_json
    from services.xslt_transformer import transform_to_html
    from configs.config import *
    
    print("✅ Tous les modules ont été importés avec succès !")
except ImportError as e:
    print(f"❌ Erreur d'import : {e}")
    print("💡 Assurez-vous d'exécuter ce notebook depuis le répertoire racine du projet")

### Vérification des Fichiers et Dossiers

In [ ]:
# Vérifier la structure du projet
print("📂 Structure du projet :\n")

directories = [
    "configs",
    "DB",
    "services",
    "data/input",
    "data/output"
]

for directory in directories:
    path = Path(directory)
    status = "✅" if path.exists() else "❌"
    print(f"{status} {directory}")

print("\n📄 Fichiers importants :\n")

files = [
    "data/input/high_popularity_spotify_data.csv",
    "data/input/spotify_transform.xslt",
    "data/input/spotify_to_json.xslt",
    "main.py",
    "requirements.txt"
]

for file in files:
    path = Path(file)
    status = "✅" if path.exists() else "❌"
    print(f"{status} {file}")

## 4. Pipeline 1 : CSV → Oracle → XML → HTML <a id="pipeline1"></a>

Ce pipeline illustre l'ensemble du processus de transformation des données, depuis leur état brut jusqu'à leur présentation web. Nous détaillons ci-après les quatre étapes principales : extraction et normalisation du CSV, insertion dans Oracle, export en XML, et transformation XSLT vers HTML.

### Étape 1.1 : Extraction et Normalisation des Données CSV

Le code ci-dessous assure l'extraction des données depuis le fichier CSV source et leur normalisation en vue de l'insertion dans la base de données.

In [ ]:
print("🔄 Extraction des données CSV...\n")

# Prétraiter le CSV
data_to_insert = preprocess_csv()

if data_to_insert:
    print("\n✅ Données extraites et normalisées !\n")
    print("📊 Aperçu des tables créées :\n")
    
    for table_name, df in data_to_insert.items():
        print(f"  • {table_name:<25} : {len(df):>6} lignes")
else:
    print("❌ Échec de l'extraction")

### Visualisation des Données Extraites

In [ ]:
# Afficher un échantillon de chaque table
if data_to_insert:
    print("📋 Échantillons de données :\n")
    
    # Genres
    print("\n=== GENRES ===")
    display(data_to_insert['sp_genres'].head(5))
    
    # Playlists
    print("\n=== PLAYLISTS ===")
    display(data_to_insert['sp_playlists'].head(5))
    
    # Tracks
    print("\n=== TRACKS ===")
    display(data_to_insert['sp_tracks'].head(5))
    
    # Audio Features
    print("\n=== AUDIO FEATURES ===")
    display(data_to_insert['sp_audio_features'].head(5))

### Étape 1.2 : Test de Connexion à Oracle

Avant toute opération sur la base de données, il est essentiel de vérifier la connectivité avec le serveur Oracle. Cette étape teste les paramètres de connexion (utilisateur, mot de passe, DSN) et s'assure que la base de données est accessible et opérationnelle.

In [ ]:
print("🔌 Test de connexion à Oracle...\n")

db_manager = DatabaseManager()

if db_manager.connect():
    print("\n✅ Connexion Oracle établie !")
    
    # Afficher les statistiques si des tables existent
    try:
        stats = db_manager.get_statistics()
        if stats:
            print("\n📊 Tables existantes :")
            for table, count in stats.items():
                print(f"  • {table:<25} : {count:>6} lignes")
    except:
        print("\nℹ️  Aucune table existante (base vide)")
    
    db_manager.close()
else:
    print("❌ Échec de la connexion Oracle")
    print("💡 Vérifiez que Oracle est démarré et les identifiants sont corrects")

### Étape 1.3 : Insertion des Données dans Oracle

Une fois les données normalisées et structurées, elles sont insérées dans la base de données Oracle en respectant l'ordre des dépendances et l'intégrité référentielle. Le module `db_manager.py` gère la connexion à la base de données, la création des tables et l'insertion sécurisée des données avec gestion automatique des clés étrangères.

In [ ]:

print("🔄 Initialisation de la base de données Oracle...\n")

db_manager = DatabaseManager()

if db_manager.connect():
    # Initialiser (drop + create tables)
    print("⚠️  Suppression et recréation des tables...")
    db_manager.initialize_db(drop_first=True)
    
    # Insérer les données
    print("\n💾 Insertion des données...")
    success = db_manager.insert_data(data_to_insert)
    
    if success:
        print("\n✅ Données insérées avec succès !")
        
        # Afficher les statistiques
        stats = db_manager.get_statistics()
        if stats:
            print("\n📊 Statistiques finales :")
            for table, count in stats.items():
                print(f"  • {table:<25} : {count:>6} lignes")
    else:
        print("❌ Échec de l'insertion")
    
    db_manager.close()

print("ℹ️  Cellule commentée pour éviter la suppression accidentelle des données")
print("💡 Décommentez le code pour exécuter l'insertion")

### Étape 1.4 : Export vers XML

Une fois les données stockées dans Oracle, elles sont extraites et exportées au format XML structuré. Cette transformation permet d'obtenir une représentation hiérarchique des données, facilitant leur échange et leur transformation ultérieure vers d'autres formats.

In [ ]:
print("🔄 Export des données vers XML...\n")

db_manager = DatabaseManager()

if db_manager.connect():
    # Récupérer les données pour l'export
    xml_data = db_manager.fetch_data_for_xml()
    
    if xml_data:
        print(f"✅ {len(xml_data)} enregistrements récupérés\n")
        
        # Exporter vers XML
        xml_file = export_to_xml(xml_data)
        
        if xml_file:
            print(f"\n✅ Fichier XML généré : {xml_file}")
            
            # Afficher la taille du fichier
            file_size = Path(xml_file).stat().st_size
            print(f"📦 Taille : {file_size:,} bytes ({file_size/1024:.2f} KB)")
    else:
        print("❌ Aucune donnée à exporter")
    
    db_manager.close()
else:
    print("❌ Échec de la connexion Oracle")

### Étape 1.5 : Validation DTD

Après la génération du fichier XML, il est essentiel de valider sa structure et sa conformité. La DTD (Document Type Definition) définit les règles de structure du XML et permet de vérifier que le document généré respecte bien le modèle attendu, garantissant ainsi l'intégrité et la cohérence des données exportées.

In [ ]:
print("🔄 Création et validation DTD...\n")

# Créer la DTD
dtd_file = create_spotify_dtd()

if dtd_file:
    print("\n✅ DTD créée avec succès\n")
    
    # Valider le XML
    xml_file = XML_OUTPUT_PATH
    
    if Path(xml_file).exists():
        is_valid, errors = validate_xml_with_dtd(xml_file, dtd_file)
        
        if is_valid:
            print("\n✅ Validation DTD réussie !")
        else:
            print(f"\n❌ Validation échouée : {len(errors)} erreur(s)")
    else:
        print(f"❌ Fichier XML introuvable : {xml_file}")
else:
    print("❌ Échec de la création de la DTD")

### Étape 1.6 : Transformation XSLT → HTML

La dernière étape du pipeline consiste à transformer le fichier XML validé en une page HTML interactive et visuellement enrichie. La feuille de style XSLT définit les règles de transformation permettant de convertir les données structurées en un format web lisible et esthétique, prêt à être consulté dans un navigateur.

In [ ]:
print("🔄 Transformation XSLT vers HTML...\n")

xml_file = XML_OUTPUT_PATH

if Path(xml_file).exists():
    html_file = transform_to_html(xml_file)
    
    if html_file:
        print(f"\n✅ Dashboard HTML généré : {html_file}")
        
        # Afficher la taille
        file_size = Path(html_file).stat().st_size
        print(f"📦 Taille : {file_size:,} bytes ({file_size/1024:.2f} KB)")
        
        print("\n💡 Ouvrez le fichier dans votre navigateur pour voir le dashboard")
    else:
        print("❌ Échec de la transformation HTML")
else:
    print(f"❌ Fichier XML introuvable : {xml_file}")

---

## 5. Pipeline 2 : XML → XSD → JSON → MongoDB <a id="pipeline2"></a>

Ce second pipeline démontre une approche alternative de gestion des données, orientée vers les bases de données NoSQL. À partir du fichier XML généré précédemment, nous mettons en œuvre une chaîne de transformation complète : validation de la structure avec un schéma XSD, conversion au format JSON, puis stockage dans une base de données MongoDB. Ce pipeline illustre l'interopérabilité des formats de données et la flexibilité offerte par les bases de données documentaires pour le stockage et l'exploitation de données semi-structurées.

### Étape 2.1 : Génération du Schéma 

Le schéma XSD (XML Schema Definition) constitue une alternative plus puissante et expressive que la DTD pour définir la structure d'un document XML. Cette étape génère automatiquement un schéma XSD à partir du fichier XML, permettant de spécifier avec précision les types de données, les contraintes et les validations applicables à chaque élément.

In [ ]:
print("🔄 Génération du schéma XSD...\n")

xsd_file = XSD_PATH

success = create_spotify_xsd(xsd_file)

if success:
    print("\n✅ Schéma XSD créé avec succès !")
    
    # Afficher la taille
    file_size = Path(xsd_file).stat().st_size
    print(f"📦 Taille : {file_size:,} bytes ({file_size/1024:.2f} KB)")
else:
    print("❌ Échec de la création du XSD")

### Étape 2.2 : Validation XSD

Une fois le schéma XSD généré, il est nécessaire de valider le fichier XML contre ce schéma pour garantir sa conformité. Cette validation vérifie que toutes les contraintes de types, de cardinalités et de structures définies dans le XSD sont respectées, assurant ainsi l'intégrité et la qualité des données avant leur conversion.

In [ ]:
print("🔄 Validation du XML avec le schéma XSD...\n")

xml_file = XML_OUTPUT_PATH
xsd_file = XSD_PATH

if Path(xml_file).exists() and Path(xsd_file).exists():
    is_valid, errors = validate_xml_with_xsd(xml_file, xsd_file)
    
    if is_valid:
        print("\n✅ Validation XSD réussie !")
        print("📋 Le XML est conforme au schéma XSD")
    else:
        print(f"\n❌ Validation échouée : {len(errors)} erreur(s)")
        for error in errors[:5]:  # Afficher les 5 premières erreurs
            print(f"  • Ligne {error.get('line', 'N/A')} : {error.get('message', 'N/A')}")
else:
    if not Path(xml_file).exists():
        print(f"❌ Fichier XML introuvable : {xml_file}")
    if not Path(xsd_file).exists():
        print(f"❌ Fichier XSD introuvable : {xsd_file}")

### Étape 2.3 : Transformation XML → JSON via 

Pour permettre le stockage des données dans MongoDB, il est nécessaire de convertir le format XML en JSON, format natif des bases de données NoSQL documentaires. Cette transformation est réalisée à l'aide d'une feuille de style XSLT spécifique qui convertit la structure hiérarchique XML en objets JSON compatibles avec MongoDB.

In [ ]:
print("🔄 Transformation XML → JSON via XSLT...\n")

xml_file = XML_OUTPUT_PATH
xslt_file = XSLT_JSON_PATH
json_file = JSON_OUTPUT_PATH

if Path(xml_file).exists() and Path(xslt_file).exists():
    success, json_data = convert_xml_to_json(xml_file, xslt_file, json_file)
    
    if success and json_data:
        print("\n✅ Conversion JSON réussie !")
        
        # Afficher des statistiques
        print(f"\n📊 Statistiques du JSON :")
        print(f"  • Total playlists    : {json_data.get('total_playlists', 0)}")
        print(f"  • Total tracks       : {json_data.get('total_tracks', 0)}")
        
        # Taille du fichier
        file_size = Path(json_file).stat().st_size
        print(f"  • Taille fichier     : {file_size:,} bytes ({file_size/1024:.2f} KB)")
    else:
        print("❌ Échec de la conversion JSON")
else:
    if not Path(xml_file).exists():
        print(f"❌ Fichier XML introuvable : {xml_file}")
    if not Path(xslt_file).exists():
        print(f"❌ Fichier XSLT introuvable : {xslt_file}")

### Étape 2.4 : Test de Connexion 


Avant d'insérer les données converties en JSON, il est indispensable de vérifier la connectivité avec le serveur MongoDB. Cette étape teste les paramètres de connexion et s'assure que la base de données NoSQL est accessible et prête à recevoir les documents JSON.

In [ ]:
print("🔌 Test de connexion à MongoDB...\n")

mongo_manager = MongoDBManager(
    host=MONGO_HOST,
    port=MONGO_PORT,
    database=MONGO_DATABASE
)

if mongo_manager.connect():
    print("\n✅ Connexion MongoDB établie !")
    
    # Lister les collections existantes
    try:
        collections = mongo_manager.db.list_collection_names()
        if collections:
            print("\n📊 Collections existantes :")
            for collection in collections:
                count = mongo_manager.db[collection].count_documents({})
                print(f"  • {collection:<20} : {count:>6} documents")
        else:
            print("\nℹ️  Aucune collection existante (base vide)")
    except Exception as e:
        print(f"⚠️  Erreur : {e}")
    
    mongo_manager.close()
else:
    print("❌ Échec de la connexion MongoDB")
    print(f"💡 Vérifiez que MongoDB est démarré sur {MONGO_HOST}:{MONGO_PORT}")

### Étape 2.5 : Insertion des Données dans MongoDB


Une fois la connexion établie et les données converties au format JSON, elles sont insérées dans MongoDB sous forme de documents. Cette étape exploite la flexibilité du modèle NoSQL pour stocker les données semi-structurées de manière efficace, permettant des requêtes rapides et une scalabilité optimale.

In [ ]:
print("🔄 Insertion des données dans MongoDB...\n")

json_file = JSON_OUTPUT_PATH

if not Path(json_file).exists():
    print(f"❌ Fichier JSON introuvable : {json_file}")
else:
    # Charger le JSON
    with open(json_file, 'r', encoding='utf-8') as f:
        json_data = json.load(f)
    
    # Se connecter à MongoDB
    mongo_manager = MongoDBManager(
        host=MONGO_HOST,
        port=MONGO_PORT,
        database=MONGO_DATABASE
    )
    
    if mongo_manager.connect():
        # Insérer les playlists (avec suppression de la collection existante)
        success, count = mongo_manager.insert_spotify_playlists(json_data, clear_first=True)
        
        if success:
            print(f"\n✅ {count} playlists insérées avec succès !")
            
            # Afficher les statistiques
            stats = mongo_manager.get_collection_stats('playlists')
        else:
            print("❌ Échec de l'insertion")
        
        mongo_manager.close()
    else:
        print("❌ Échec de la connexion MongoDB")

## 6. Dashboard de Visualisation <a id="dashboard"></a>

Pour faciliter l'exploration et l'analyse des données Spotify, nous avons développé une application web interactive permettant de visualiser les résultats des deux pipelines. Ce dashboard, développé avec Node.js (backend) et React.js (frontend), s'appuie sur MongoDB pour offrir une interface dynamique et responsive.

### Architecture du Dashboard

L'application se compose de trois éléments principaux :
- **Backend (Node.js)** : API REST exposant les données sur le port 5000
- **Frontend (React.js)** : Interface utilisateur moderne et interactive sur le port 3000
- **MongoDB** : Base de données NoSQL stockant les données converties en JSON

### Lancement du Dashboard


In [ ]:
# Chemin vers le dossier spotify-dashboard
dashboard_path = Path("spotify-dashboard")

print("="*50)
print("   Installation et lancement du Dashboard")
print("="*50)
print()

# Étape 1 : Installation des dépendances
print("[1/3] Installation des dépendances...")
print()

# Installation backend
print("Installation du backend...")
backend_path = dashboard_path / "backend"
result = subprocess.run(["npm", "install"], cwd=backend_path, shell=True, capture_output=True, text=True)
if result.returncode == 0:
    print("✅ Backend installé avec succès")
else:
    print(f"❌ Erreur backend : {result.stderr}")

# Installation frontend
print("Installation du frontend...")
frontend_path = dashboard_path / "frontend"
result = subprocess.run(["npm", "install"], cwd=frontend_path, shell=True, capture_output=True, text=True)
if result.returncode == 0:
    print("✅ Frontend installé avec succès")
else:
    print(f"❌ Erreur frontend : {result.stderr}")

print()
print("="*50)
print("   Installation terminée")
print("="*50)
print()

# Lancement backend (en arrière-plan)
print("Démarrage du backend sur http://localhost:5000")
subprocess.Popen(["npm", "run", "dev"], cwd=backend_path, shell=True)

# Attendre 3 secondes
import time
time.sleep(3)

# Lancement frontend (en arrière-plan)
print("Démarrage du frontend sur http://localhost:3200")
subprocess.Popen(["npm", "run", "dev"], cwd=frontend_path, shell=True)

print()
print("="*50)
print("   Dashboard lancé avec succès!")
print("="*50)
print()
print("🌐 Backend:  http://localhost:5000")
print("🌐 Frontend: http://localhost:3200")

## Démarrage manuel du Dashboard

### Démarrage du Backend
```bash
cd spotify-dashboard/backend
npm run dev
```

Le serveur backend démarre sur **http://localhost:5000**

---

### Démarrage du Frontend
```bash
cd spotify-dashboard/frontend
npm run dev
```

L'interface frontend est accessible sur **http://localhost:3200**


## 7. Conclusion <a id="conclusion"></a>

### Synthèse du projet

Ce projet a permis de mettre en œuvre deux pipelines complets de traitement de données, illustrant concrètement les processus d'extraction, transformation, stockage et présentation de données. Du fichier CSV brut jusqu'aux visualisations web interactives, nous avons démontré l'ensemble du cycle de vie des données en exploitant des technologies complémentaires.

### Objectifs atteints

Le **premier pipeline (CSV → Oracle → XML → HTML)** a démontré l'approche relationnelle avec :
- La normalisation complète des données brutes (8 tables en 3NF)
- La gestion de l'intégrité référentielle dans Oracle
- La validation structurelle avec DTD
- La transformation XSLT pour la présentation web

Le **second pipeline (XML → XSD → JSON → MongoDB)** a illustré l'approche NoSQL avec :
- La validation avancée par schéma XSD
- La conversion de formats (XML vers JSON)
- Le stockage flexible dans MongoDB
- L'exploitation des données via une API REST

Le **dashboard de visualisation** a permis de :
- Créer une interface web moderne et interactive
- Exploiter les données stockées dans MongoDB
- Offrir une expérience utilisateur fluide et intuitive

### Compétences acquises

Ce projet a permis de maîtriser :
- La **modélisation de données** relationnelles et NoSQL
- La **normalisation** et l'optimisation de schémas
- La **manipulation de données** avec Python (pandas, oracledb)
- Les **formats d'échange** standardisés (XML, JSON)
- Les **technologies de validation** (DTD, XSD)
- Les **transformations de données** (XSLT)
- Le **développement full-stack** (Node.js, React.js)
- L'**intégration de systèmes** hétérogènes

### Bilan

Ce projet a démontré qu'une approche méthodique et structurée permet de transformer efficacement des données brutes en informations exploitables et visuellement enrichies. L'utilisation complémentaire de bases de données relationnelles (Oracle) et NoSQL (MongoDB) a illustré la flexibilité nécessaire dans les architectures modernes de gestion de données. 

Les compétences acquises constituent une base solide pour appréhender les problématiques réelles de traitement et d'exploitation de données dans des environnements professionnels.


## Lien github du projet

**https://github.com/kdcl3110/DBA_Spotify**

